In [1]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF(r"C:\Users\srika\OneDrive\Desktop\Fall19\DeepLearningForNLP\Project\Datasets\Amazon_with_data\reviews_Toys_and_Games_5.json.gz")

In [2]:
len(df["asin"].unique())

11924

In [3]:
len(df)

167597

In [4]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1VXOAVRGKGEAK,0439893577,Angie,"[0, 0]",I like the item pricing. My granddaughter want...,5.0,Magnetic board,1390953600,"01 29, 2014"
1,A8R62G708TSCM,0439893577,Candace,"[1, 1]",Love the magnet easel... great for moving to d...,4.0,it works pretty good for moving to different a...,1395964800,"03 28, 2014"
2,A21KH420DK0ICA,0439893577,capemaychristy,"[1, 1]",Both sides are magnetic. A real plus when you...,5.0,love this!,1359331200,"01 28, 2013"
3,AR29QK6HPFYZ4,0439893577,dcrm,"[0, 0]",Bought one a few years ago for my daughter and...,5.0,Daughters love it,1391817600,"02 8, 2014"
4,ACCH8EOML6FN5,0439893577,DoyZ,"[1, 1]",I have a stainless steel refrigerator therefor...,4.0,Great to have so he can play with his alphabet...,1399248000,"05 5, 2014"


In [5]:
df.describe()

,overall,unixReviewTime
count,167597.000000,1.675970e+05
mean,4.356307,1.348759e+09
std,0.993501,6.089581e+07
min,1.000000,9.647424e+08
25%,4.000000,1.335917e+09
50%,5.000000,1.364429e+09
75%,5.000000,1.388016e+09
max,5.000000,1.406074e+09


In [6]:
import nltk.classify.util
#from nltk.classify import NaiveBayesClassifier
import numpy as np
import re
import string
import nltk

cleanup_re = re.compile('[^a-z]+')
def cleanup(sentence):
    sentence = str(sentence)
    sentence = sentence.lower()
    sentence = cleanup_re.sub(' ', sentence).strip()
    #sentence = " ".join(nltk.word_tokenize(sentence))
    return sentence

df["Summary_Clean"] = df["summary"].apply(cleanup)
df["reviewText_Clean"] = df["reviewText"].apply(cleanup)

In [7]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,Summary_Clean,reviewText_Clean
0,A1VXOAVRGKGEAK,0439893577,Angie,"[0, 0]",I like the item pricing. My granddaughter want...,5.0,Magnetic board,1390953600,"01 29, 2014",magnetic board,i like the item pricing my granddaughter wante...
1,A8R62G708TSCM,0439893577,Candace,"[1, 1]",Love the magnet easel... great for moving to d...,4.0,it works pretty good for moving to different a...,1395964800,"03 28, 2014",it works pretty good for moving to different a...,love the magnet easel great for moving to diff...
2,A21KH420DK0ICA,0439893577,capemaychristy,"[1, 1]",Both sides are magnetic. A real plus when you...,5.0,love this!,1359331200,"01 28, 2013",love this,both sides are magnetic a real plus when you r...
3,AR29QK6HPFYZ4,0439893577,dcrm,"[0, 0]",Bought one a few years ago for my daughter and...,5.0,Daughters love it,1391817600,"02 8, 2014",daughters love it,bought one a few years ago for my daughter and...
4,ACCH8EOML6FN5,0439893577,DoyZ,"[1, 1]",I have a stainless steel refrigerator therefor...,4.0,Great to have so he can play with his alphabet...,1399248000,"05 5, 2014",great to have so he can play with his alphabet,i have a stainless steel refrigerator therefor...


In [8]:
#Group by Product
tempDf = df[['asin','Summary_Clean']]

In [9]:
#New Dataframe with Group by ProductID
newdf = tempDf.groupby(['asin'])['Summary_Clean'].apply(lambda x: ". ".join(x.astype(str))).reset_index()

In [10]:
#Check the DataFrame
#Summary for each individual product
newdf.head()

,asin,Summary_Clean
0,0439893577,magnetic board. it works pretty good for movin...
1,048645195X,intricate. creative soothing and inspirational...
2,0545496470,book of impossible objects. lots of stuff. gre...
3,0615444172,good book. two year old loves this. great stic...
4,0670010936,llama llama cuddly. love these books. not so c...


In [11]:
#Check whether each summary got combined?
newdf.iloc[0,1]

'magnetic board. it works pretty good for moving to different areas. love this. daughters love it. great to have so he can play with his alphabet. grandma. serves the purpose. great. a must have for any homeschooling classroom. nifty little thing. works great. perfect magnetic board. not very durable. great magnet board. every child should have this. quality product. great size'

In [12]:
#Creating temporary DF for MCS Score
temp_df=df

In [13]:
#Creating variables to find MCS score
#import numpy as np
from tqdm import tqdm_notebook
df['help'] = df.apply(lambda row:int(row.helpful[0]), axis = 1)

In [14]:
#Smoothing the data
df['augumented'] = df.apply(lambda row: int(row.overall)*(row.help+1),axis = 1)

In [15]:
#Smoothing the data
df['help'] = df.apply(lambda row:row.help+1,axis =1)

In [16]:
#Check the dataframe
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,Summary_Clean,reviewText_Clean,help,augumented
0,A1VXOAVRGKGEAK,0439893577,Angie,"[0, 0]",I like the item pricing. My granddaughter want...,5.0,Magnetic board,1390953600,"01 29, 2014",magnetic board,i like the item pricing my granddaughter wante...,1,5
1,A8R62G708TSCM,0439893577,Candace,"[1, 1]",Love the magnet easel... great for moving to d...,4.0,it works pretty good for moving to different a...,1395964800,"03 28, 2014",it works pretty good for moving to different a...,love the magnet easel great for moving to diff...,2,8
2,A21KH420DK0ICA,0439893577,capemaychristy,"[1, 1]",Both sides are magnetic. A real plus when you...,5.0,love this!,1359331200,"01 28, 2013",love this,both sides are magnetic a real plus when you r...,2,10
3,AR29QK6HPFYZ4,0439893577,dcrm,"[0, 0]",Bought one a few years ago for my daughter and...,5.0,Daughters love it,1391817600,"02 8, 2014",daughters love it,bought one a few years ago for my daughter and...,1,5
4,ACCH8EOML6FN5,0439893577,DoyZ,"[1, 1]",I have a stainless steel refrigerator therefor...,4.0,Great to have so he can play with his alphabet...,1399248000,"05 5, 2014",great to have so he can play with his alphabet,i have a stainless steel refrigerator therefor...,2,8


In [17]:
#Reset the Index
ttdf = df.groupby(['asin'])['augumented'].apply(lambda x: sum(x)).reset_index()

In [18]:
#Reset the Index
tt1df = df.groupby(['asin'])['help'].apply(lambda x: sum(x)).reset_index()

In [19]:
#Attaching help for tt1df
ttdf['help'] = tt1df['help'] 

In [20]:
#Check the DataFrame
ttdf.head()

,asin,augumented,help
0,0439893577,379,102
1,048645195X,170,38
2,0545496470,48,15
3,0615444172,60,12
4,0670010936,56,14


In [21]:
#Calculating MCS Score
ttdf['MCS'] = ttdf.apply(lambda row:float(row.augumented/row.help),axis = 1)

In [22]:
#Attaching for DataFrame
newdf["MCS"]=ttdf["MCS"]

In [23]:
#Assigning for final DataFrame
Finaldf=newdf

In [24]:
#Checking head for DataFrame
Finaldf.head()

,asin,Summary_Clean,MCS
0,0439893577,magnetic board. it works pretty good for movin...,3.715686
1,048645195X,intricate. creative soothing and inspirational...,4.473684
2,0545496470,book of impossible objects. lots of stuff. gre...,3.200000
3,0615444172,good book. two year old loves this. great stic...,5.000000
4,0670010936,llama llama cuddly. love these books. not so c...,4.000000


In [27]:
#Truncating the value for MCS
Finaldf['MCS']=Finaldf['MCS'].apply(lambda x: round(x,2))


In [28]:
Finaldf.head()

,asin,Summary_Clean,MCS
0,0439893577,magnetic board. it works pretty good for movin...,3.72
1,048645195X,intricate. creative soothing and inspirational...,4.47
2,0545496470,book of impossible objects. lots of stuff. gre...,3.20
3,0615444172,good book. two year old loves this. great stic...,5.00
4,0670010936,llama llama cuddly. love these books. not so c...,4.00


In [29]:
#Length of Unique products
len(Finaldf)

11924

In [30]:
#Check whether the summary got concatenated
Finaldf.iloc[0,1]

'magnetic board. it works pretty good for moving to different areas. love this. daughters love it. great to have so he can play with his alphabet. grandma. serves the purpose. great. a must have for any homeschooling classroom. nifty little thing. works great. perfect magnetic board. not very durable. great magnet board. every child should have this. quality product. great size'